# Visualize the molecule

In [126]:
from rdkit import Chem
import py3Dmol
mol_folder = './Molecules/'
mol_fname = '25A12Rot_37.mol2'
mol_filepath = mol_folder + mol_fname
mol = Chem.MolFromMol2File(mol_filepath)
mblock = Chem.MolToMolBlock(mol)
view = py3Dmol.view(width=500, height=250)
view.addModel(mblock, 'mol')
view.setStyle({'stick':{}})
view.zoomTo()
view.show()


You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

# Input Variables

In [127]:
n_atoms_mol = mol.GetNumAtoms()  # no. of atoms in the molecule (excluding hydrogen atoms)
print('No. of atoms: ', n_atoms_mol)
conformers = mol.GetConformers()
conf = conformers[0]
coords_mol = {}  # Coordinates of the atoms
for i in range(n_atoms_mol):
    coords_mol[i] = list(conf.GetAtomPosition(i))
print('Coordinates of atoms:')
for i in coords_mol:
    print(i, '\t:', coords_mol[i])

No. of atoms:  25
Coordinates of atoms:
0 	: [10.932, -2.277, -2.406]
1 	: [9.921, -1.254, -1.913]
2 	: [9.095, -1.798, -0.745]
3 	: [8.087, -0.752, -0.246]
4 	: [7.183, -1.278, 0.798]
5 	: [7.885, -1.44, 2.071]
6 	: [6.007, -0.379, 0.923]
7 	: [4.858, -0.861, 1.855]
8 	: [5.052, -0.424, 3.26]
9 	: [3.509, -0.292, 1.355]
10 	: [2.453, -0.812, 2.189]
11 	: [1.249, -0.271, 1.824]
12 	: [0.893, 0.979, 2.332]
13 	: [-0.354, 1.522, 2.027]
14 	: [-1.251, 0.813, 1.223]
15 	: [-0.901, -0.442, 0.715]
16 	: [-1.754, -1.194, -0.047]
17 	: [-2.456, -0.465, -1.059]
18 	: [-2.965, -1.395, -2.123]
19 	: [-2.144, -2.566, -2.605]
20 	: [-3.401, -2.806, -1.811]
21 	: [0.344, -0.99, 1.04]
22 	: [0.723, -2.314, 0.53]
23 	: [1.527, -2.404, -0.393]
24 	: [0.085, -3.505, 1.189]


In [128]:
input_folder = './Input_Rot_2/'
input_fname = mol_fname[:-5] + '_input.txt'
input_filepath = input_folder + input_fname
input_lines = open(input_filepath, 'r').readlines()
torsional_bonds_mol = eval(input_lines[0])
median_list_mol = []
for atoms in input_lines[1].split(','):
    median_list_mol.append(int(atoms))
coords_rotation_mol = eval(input_lines[2])
distance_pairs_mol = eval(input_lines[3])
print(torsional_bonds_mol)
print(median_list_mol)
print(coords_rotation_mol)
print(distance_pairs_mol)

{0: (6, 8), 1: (15, 16)}
[2, 5, 6, 7, 8, 12, 15, 16, 19, 22, 23]
{9: [0], 10: [0], 11: [0], 12: [0], 13: [0], 14: [0], 15: [0], 16: [0], 17: [0, 1], 18: [0, 1], 19: [0, 1], 20: [0], 21: [0], 22: [0], 23: [0]}
{2: [12, 15, 16, 19, 22, 23], 5: [12, 15, 16, 19, 22, 23], 6: [12, 15, 16, 19, 22, 23], 7: [12, 15, 16, 19, 22, 23], 8: [12, 15, 16, 19, 22, 23], 12: [19], 15: [19], 16: [], 19: [22, 23], 22: [], 23: []}


In [129]:
# torsional_bonds_mol = { 0: (9, 10), 1: (19, 20)  }  # Torsional bond numbers and their respective coordinate ends
# median_list_mol = [2, 6, 9, 10, 14, 19, 20, 23]  # List of atoms (median) considered for the HUBO construction
# coords_rotation_mol = {11:[0], 12:[0], 13:[0], 14:[0], 15:[0], 16:[0], 17:[0], 18:[0], 19:[0], 20:[0], 21:[0,1], 22:[0,1], 23:[0,1], 24:[0,1] }  # atom index and the rotatable bond numbers list that affect them
# distance_pairs_mol = {2:[14, 19, 20, 23], 6:[14, 19, 20, 23], 9:[14, 19, 20, 23], 10:[23], 14:[23], 19:[23], 20:[], 23:[] }  # indices dictionary to find distances

In [130]:
import copy

coords_median = {}
for i, index in enumerate(median_list_mol):
    coords_median[i] = coords_mol[index]
coords_median_2 = copy.deepcopy(coords_median)
print('coords_median: ', coords_median)

torsional_bonds = {}
for i in torsional_bonds_mol:
    torsional_bonds[i] = (median_list_mol.index(torsional_bonds_mol[i][0]), median_list_mol.index(torsional_bonds_mol[i][1]))
print('\ntorsional_bonds: ', torsional_bonds)

coords_rotation_dict = {}
for atom in median_list_mol:
    if atom in coords_rotation_mol.keys():
        coords_rotation_dict[median_list_mol.index(atom)] = coords_rotation_mol[atom]
print('\ncoords_rotation_dict: ', coords_rotation_dict)

distance_pairs = []
for x, y in distance_pairs_mol.items():
    u = median_list_mol.index(x)
    for i in y:
        v = median_list_mol.index(i)
        distance_pairs.append((u, v))
print('\ndistance pairs: ', distance_pairs)



coords_median:  {0: [9.095, -1.798, -0.745], 1: [7.885, -1.44, 2.071], 2: [6.007, -0.379, 0.923], 3: [4.858, -0.861, 1.855], 4: [5.052, -0.424, 3.26], 5: [0.893, 0.979, 2.332], 6: [-0.901, -0.442, 0.715], 7: [-1.754, -1.194, -0.047], 8: [-2.144, -2.566, -2.605], 9: [0.723, -2.314, 0.53], 10: [1.527, -2.404, -0.393]}

torsional_bonds:  {0: (2, 4), 1: (6, 7)}

coords_rotation_dict:  {5: [0], 6: [0], 7: [0], 8: [0, 1], 9: [0], 10: [0]}

distance pairs:  [(0, 5), (0, 6), (0, 7), (0, 8), (0, 9), (0, 10), (1, 5), (1, 6), (1, 7), (1, 8), (1, 9), (1, 10), (2, 5), (2, 6), (2, 7), (2, 8), (2, 9), (2, 10), (3, 5), (3, 6), (3, 7), (3, 8), (3, 9), (3, 10), (4, 5), (4, 6), (4, 7), (4, 8), (4, 9), (4, 10), (5, 8), (6, 8), (8, 9), (8, 10)]


In [131]:
import sympy as sp
n_bonds = len(torsional_bonds)  # no. of bonds
n_angles = 8  # no. of discrete angles
x = sp.symbols(f'x(0:{n_bonds*n_angles})')  # hubo variables
print(x)

(x0, x1, x2, x3, x4, x5, x6, x7, x8, x9, x10, x11, x12, x13, x14, x15)


# Function to generate hard constraint

In [132]:
# generate hard constraint
def generate_hard_hubo():
    hard_constraint = 0
    index = 0
    for i in range(n_bonds):
        summation = 0
        for j in range(n_angles):
            summation += x[index]
            index += 1
        hard_constraint += (summation - 1) ** 2
    a_const = sp.Symbol('A_const')
    hard_constraint *= a_const
    return hard_constraint

# Functions to rotate coordinates with hubo variables

In [133]:
def rotate_coordinates(rotation_matrix, old_coords):
    coord_vector = ones(4, 1)
    coord_vector[0, 0] = old_coords[0]
    coord_vector[1, 0] = old_coords[1]
    coord_vector[2, 0] = old_coords[2]
    coord_rot_vector = sp.expand(rotation_matrix * coord_vector)
    return [coord_rot_vector[0, 0], coord_rot_vector[1, 0], coord_rot_vector[2, 0]]

In [134]:
def generate_thetas():
    angle_incr = 2 * sp.pi / n_angles
    thetas = [i*angle_incr for i in range(n_angles)]
    return thetas

In [135]:
def rotation_matrix_hubo(first_coords, second_coords, bond_no):
    x_dash, y_dash, z_dash = first_coords[0], first_coords[1], first_coords[2]
    x_ddash, y_ddash, z_ddash = second_coords[0], second_coords[1], second_coords[2]
    dx = x_ddash - x_dash
    dy = y_ddash - y_dash
    dz = z_ddash - z_dash
    l_sq = dx ** 2 + dy ** 2 + dz ** 2
    l = sp.sqrt(l_sq)
    thetas = generate_thetas()
    c_theta = 0.0
    s_theta = 0.0
    index = n_angles * bond_no
    for i in range(n_angles):
        c_theta += sp.cos(thetas[i]) * x[index]
        s_theta += sp.sin(thetas[i]) * x[index]
        index += 1
    rotation_matrix = eye(4)
    rotation_matrix[0, 0] = (dx ** 2 + (dy ** 2 + dz ** 2) * c_theta) / l_sq
    rotation_matrix[0, 1] = (dx * dy * (1 - c_theta) - dz * l * s_theta) / l_sq
    rotation_matrix[0, 2] = (dx * dz * (1 - c_theta) + dy * l * s_theta) / l_sq
    rotation_matrix[0, 3] = ((x_dash * (dy ** 2 + dz ** 2) - dx * (y_dash * dy + z_dash * dz)) * (1 - c_theta) + (
            y_dash * dz - z_dash * dy) * l * s_theta) / l_sq
    rotation_matrix[1, 0] = (dx * dy * (1 - c_theta) + dz * l * s_theta) / l_sq
    rotation_matrix[1, 1] = (dy ** 2 + (dx ** 2 + dz ** 2) * c_theta) / l_sq
    rotation_matrix[1, 2] = (dy * dz * (1 - c_theta) - dx * l * s_theta) / l_sq
    rotation_matrix[1, 3] = ((y_dash * (dx ** 2 + dz ** 2) - dy * (x_dash * dx + z_dash * dz)) * (1 - c_theta) + (
            z_dash * dx - x_dash * dz) * l * s_theta) / l_sq
    rotation_matrix[2, 0] = (dx * dz * (1 - c_theta) - dy * l * s_theta) / l_sq
    rotation_matrix[2, 1] = (dy * dz * (1 - c_theta) + dx * l * s_theta) / l_sq
    rotation_matrix[2, 2] = (dz ** 2 + (dx ** 2 + dy ** 2) * c_theta) / l_sq
    rotation_matrix[2, 3] = ((z_dash * (dx ** 2 + dy ** 2) - dz * (x_dash * dx + y_dash * dy)) * (1 - c_theta) + (
            x_dash * dy - y_dash * dx) * l * s_theta) / l_sq
    return rotation_matrix

In [136]:
from sympy.matrices import ones, eye
import time
start = time.time()

for i in coords_rotation_dict:
    rot_mat = eye(4, 4)
    for bond_no in coords_rotation_dict[i]:
        temp_rot_mat = rotation_matrix_hubo(coords_median[torsional_bonds[bond_no][0]], coords_median[torsional_bonds[bond_no][1]], bond_no)
        rot_mat = rot_mat * temp_rot_mat
    coords_median_2[i] = rotate_coordinates(rot_mat, coords_median[i])

end = time.time()
print("The time of execution: ", (end-start), "seconds")

The time of execution:  5.733651399612427 seconds


# Functions to generate the optimization contraint in hubo

In [137]:
def distance_squared(first_coords, second_coords):
    dis_sq = (second_coords[0] - first_coords[0])**2 + (second_coords[1] - first_coords[1])**2 + (second_coords[2] - first_coords[2])**2
    return dis_sq

def generate_distance_hubo():
    distance_sq = 0
    for pair in distance_pairs:
        distance_sq += distance_squared(coords_median_2[pair[0]], coords_median_2[pair[1]])
    return distance_sq.expand()

In [138]:
start = time.time()

hard_constraint = generate_hard_hubo()
distance_constraint = generate_distance_hubo()
hubo_expr = sp.expand(hard_constraint - distance_constraint)

end = time.time()
print("The time of execution: ", (end-start), "seconds")

The time of execution:  139.42659521102905 seconds


# The full hubo expression is written in the file 'full_hubo_expr.txt'

In [139]:
def hubo_expr_to_dict():
    hubo_args = hubo_expr.args
    hubo_dict = {}
    for monom in hubo_args:
        dict_value = monom.as_coeff_mul()[0]
        monom_key = []
        monom_coeffs = monom.as_coeff_mul()[1]
        for monom_item in monom_coeffs:
            if re.match("^x(\d*)\d$", str(monom_item)):
                monom_key.append(int(str(monom_item)[1:]))
            elif re.match("^x(\d*)\\*\\*(\d)$", str(monom_item)):
                monom_key.append(int(str(monom_item).split('**')[0][1:]))
            else:
                dict_value *= monom_item
        if len(monom_key) > 0:
            monom_key.sort()
            dict_key = tuple(monom_key)
        else:
            dict_key = ()
        dict_value = dict_value.evalf()
        if dict_key in hubo_dict:
            hubo_dict[dict_key] += dict_value
        else:
            hubo_dict[dict_key] = dict_value
    return hubo_dict

In [140]:
import re
hubo_dict = hubo_expr_to_dict()

In [141]:
hubo_folder = './Hubo_Rot_2/'
hubo_fname = mol_fname[:-5] + '_hubo.txt'
hubo_filepath = hubo_folder + hubo_fname
f = open(hubo_filepath, "w")
print('full hubo dictionary written in file - ' + hubo_filepath)
f.write(str(hubo_dict))
f.close()

full hubo dictionary written in file - ./Hubo_Rot_2/25A12Rot_37_hubo.txt


In [151]:
#This is used to check the maximum coefficient appearing in Hubo_B
A_const=0
read_dictionary_B = open(hubo_filepath, 'r').read()
HUBO_B=eval(read_dictionary_B)

#We set the Hard constraint strength as the (maximum coefficient appearing in Hubo_B)*const.
#const was empirically selected to be 10
const=10
A_const=max(map(abs, list(HUBO_B.values())))*const
# A_const = 1000

#read the final HUBO
read_dictionary= open(hubo_filepath, 'r').read()
HUBO=eval(read_dictionary)

In [152]:
print("Current size of the HUBO:",len(HUBO)) 

Current size of the HUBO: 1369


In [153]:
def threshold_approx(h, val=1):
    d =h.copy()
    monoms = h.keys()
    for m in monoms:     
        temp = d[m]
        if (temp < 0.0):
            temp = -1.0 * temp
        if (temp <= (10.0 ** (val))):
            del d[m]
    return d

In [154]:
#Coefficints with absolute value less than 10^{threshold} are deleted from the HUBO.
threshold=3

HUBO=threshold_approx(HUBO,threshold)

print("Size of the HUBO after threshold approximation:",len(HUBO))

Size of the HUBO after threshold approximation: 73


In [155]:
import dimod
#calculate the strength parameter needed by make_quadratic
max_hubo_value=max(map(abs, list(HUBO.values())))
strength=1.5*max_hubo_value
#generate the bqm
bqm = dimod.make_quadratic(HUBO, strength, dimod.BINARY)

In [156]:
def find_bond_theta_soln(solution):
    thetas = generate_thetas()
    bond_theta_soln = {}
    for key, value in solution.items():
        if solution[key] == 1:
            bond_theta_soln[key//n_angles] = thetas[key%n_angles]
    return bond_theta_soln

In [157]:
import neal
sampler = neal.SimulatedAnnealingSampler()
sample_size=10
start = time.time()
sampleset = sampler.sample(bqm, num_reads=sample_size)
end = time.time()
print("The time of execution: ", (end-start), "s")
print(sampleset)

The time of execution:  0.016998291015625 s
   0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15       energy num_oc.
9  0  0  0  0  0  0  0  1  0  0  0  0  1  0  0  0 -2097.216642       1
5  0  0  0  0  0  0  0  1  0  0  1  0  0  0  0  0 -2069.452944       1
0  0  0  0  0  0  0  0  1  0  0  0  0  0  0  1  0 -2064.565039       1
3  0  0  0  0  0  0  1  0  0  0  0  0  0  0  0  1 -1978.321098       1
8  0  0  0  0  0  0  1  0  0  1  0  0  0  0  0  0  -1971.36733       1
4  0  0  0  0  0  0  1  0  0  0  0  0  0  0  1  0 -1969.435042       1
6  1  0  0  0  0  0  0  0  0  1  0  0  0  0  0  0 -1951.971708       1
2  0  0  0  0  0  1  0  0  0  1  0  0  0  0  0  0 -1722.307578       1
7  0  1  0  0  0  0  0  0  0  0  1  0  0  0  0  0 -1697.833642       1
1  0  0  0  0  1  0  0  0  0  0  0  0  0  1  0  0 -1475.978885       1
['BINARY', 10 rows, 10 samples, 16 variables]


In [158]:
sa_soln = sampleset.first.sample
print(sa_soln)
bond_theta_soln = find_bond_theta_soln(sa_soln)
print(bond_theta_soln)

{0: 0, 1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0, 7: 1, 8: 0, 9: 0, 10: 0, 11: 0, 12: 1, 13: 0, 14: 0, 15: 0}
{0: 7*pi/4, 1: pi}


In [159]:
from dwave.system import DWaveSampler, EmbeddingComposite
sampler = EmbeddingComposite(DWaveSampler())
start = time.time()
sampleset = sampler.sample(bqm, num_reads=1000)
end = time.time()
print("The time of execution: ", (end-start), "s")
print(sampleset.slice(10))

The time of execution:  0.20296144485473633 s
   0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15       energy num_oc. ...
0  0  0  0  0  0  0  0  1  0  0  0  0  1  0  0  0 -2097.216642       9 ...
1  0  0  0  0  0  0  0  1  0  0  0  1  0  0  0  0 -2091.128627      11 ...
2  0  0  0  0  0  0  0  1  0  0  0  0  0  1  0  0 -2077.262318       3 ...
3  0  0  0  0  0  0  0  1  1  0  0  0  0  0  0  0 -2077.104742       7 ...
4  0  0  0  0  0  0  0  1  0  0  0  0  0  0  0  1 -2073.451095      12 ...
5  0  0  0  0  0  0  0  1  0  0  1  0  0  0  0  0 -2069.452944      19 ...
6  0  0  0  0  0  0  0  1  0  1  0  0  0  0  0  0 -2066.497328      11 ...
7  0  0  0  0  0  0  0  1  0  0  0  0  0  0  1  0 -2064.565039       6 ...
8  0  0  0  0  0  0  1  0  0  0  0  0  1  0  0  0 -2002.086644      14 ...
9  0  0  0  0  0  0  1  0  0  0  0  1  0  0  0  0 -1995.998629       6 ...
['BINARY', 10 rows, 98 samples, 16 variables]


In [160]:
qa_soln = sampleset.first.sample
print(qa_soln)
bond_theta_soln = find_bond_theta_soln(qa_soln)
print(bond_theta_soln)

{0: 0, 1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0, 7: 1, 8: 0, 9: 0, 10: 0, 11: 0, 12: 1, 13: 0, 14: 0, 15: 0}
{0: 7*pi/4, 1: pi}


In [161]:
def rotation_matrix_new_coords(first_coords, second_coords, bond_no, soln_theta):
    x_dash, y_dash, z_dash = first_coords[0], first_coords[1], first_coords[2]
    x_ddash, y_ddash, z_ddash = second_coords[0], second_coords[1], second_coords[2]
    dx = x_ddash - x_dash
    dy = y_ddash - y_dash
    dz = z_ddash - z_dash
    l_sq = dx ** 2 + dy ** 2 + dz ** 2
    l = sp.sqrt(l_sq)
    c_theta = sp.cos(soln_theta)
    s_theta = sp.sin(soln_theta)
    index = n_angles * bond_no
    rotation_matrix = eye(4)
    rotation_matrix[0, 0] = ((dx ** 2 + (dy ** 2 + dz ** 2) * c_theta) / l_sq).evalf()
    rotation_matrix[0, 1] = ((dx * dy * (1 - c_theta) - dz * l * s_theta) / l_sq).evalf()
    rotation_matrix[0, 2] = ((dx * dz * (1 - c_theta) + dy * l * s_theta) / l_sq).evalf()
    rotation_matrix[0, 3] = (((x_dash * (dy ** 2 + dz ** 2) - dx * (y_dash * dy + z_dash * dz)) * (1 - c_theta) + (
            y_dash * dz - z_dash * dy) * l * s_theta) / l_sq).evalf()
    rotation_matrix[1, 0] = ((dx * dy * (1 - c_theta) + dz * l * s_theta) / l_sq).evalf()
    rotation_matrix[1, 1] = ((dy ** 2 + (dx ** 2 + dz ** 2) * c_theta) / l_sq).evalf()
    rotation_matrix[1, 2] = ((dy * dz * (1 - c_theta) - dx * l * s_theta) / l_sq).evalf()
    rotation_matrix[1, 3] = (((y_dash * (dx ** 2 + dz ** 2) - dy * (x_dash * dx + z_dash * dz)) * (1 - c_theta) + (
            z_dash * dx - x_dash * dz) * l * s_theta) / l_sq).evalf()
    rotation_matrix[2, 0] = ((dx * dz * (1 - c_theta) - dy * l * s_theta) / l_sq).evalf()
    rotation_matrix[2, 1] = ((dy * dz * (1 - c_theta) + dx * l * s_theta) / l_sq).evalf()
    rotation_matrix[2, 2] = ((dz ** 2 + (dx ** 2 + dy ** 2) * c_theta) / l_sq).evalf()
    rotation_matrix[2, 3] = (((z_dash * (dx ** 2 + dy ** 2) - dz * (x_dash * dx + y_dash * dy)) * (1 - c_theta) + (
            x_dash * dy - y_dash * dx) * l * s_theta) / l_sq).evalf()
    return rotation_matrix

In [162]:
def compute_new_coords(solution):
    final_coords = copy.deepcopy(coords_mol)
    for i in coords_rotation_mol:
        if len(coords_rotation_mol[i]) > 0:
            rot_mat = eye(4, 4)
            for bond_no in coords_rotation_mol[i]:
                temp_rot_mat = rotation_matrix_new_coords(coords_mol[torsional_bonds_mol[bond_no][0]], coords_mol[torsional_bonds_mol[bond_no][1]], bond_no, solution[bond_no])
                rot_mat = rot_mat * temp_rot_mat
            final_coords[i] = rotate_coordinates(rot_mat, coords_mol[i])
    return final_coords


In [163]:
compute_new_coords(bond_theta_soln)

{0: [10.932, -2.277, -2.406],
 1: [9.921, -1.254, -1.913],
 2: [9.095, -1.798, -0.745],
 3: [8.087, -0.752, -0.246],
 4: [7.183, -1.278, 0.798],
 5: [7.885, -1.44, 2.071],
 6: [6.007, -0.379, 0.923],
 7: [4.858, -0.861, 1.855],
 8: [5.052, -0.424, 3.26],
 9: [4.15424692230288, 1.19481223240277, 1.64730524657996],
 10: [2.94695281637116, 1.28901773337597, 2.43130669132922],
 11: [2.43310506544955, 2.55658382000679, 2.36232332452059],
 12: [2.94131784821421, 3.53455415247080, 3.21824025755488],
 13: [2.39118009841086, 4.81525518393715, 3.21221329792877],
 14: [1.32621332137866, 5.11803594690604, 2.35905705585254],
 15: [0.810224342729386, 4.14044452471101, 1.50251786517696],
 16: [-0.253789818892602, 4.37267094753617, 0.673240015231790],
 17: [-1.23078863351286, 3.32722613853197, 0.643115813055177],
 18: [-2.54289491454414, 3.84504976979816, 0.127060782868885],
 19: [-3.05435752200189, 5.21336166828141, 0.506184179194761],
 20: [-2.38693903055578, 4.78759482502551, -1.25039238641619],
 2

In [164]:
def find_volume_change(old_coords, new_coords):
    n = len(old_coords)
    old_distance_sq = 0
    for i in range(n-1):
        for j in range(i+1, n):
            old_distance_sq += distance_squared(old_coords[i], old_coords[j])
    
    new_distance_sq = 0
    for i in range(n-1):
        for j in range(i+1, n):
            new_distance_sq += distance_squared(new_coords[i], new_coords[j])
    # print("new volume: ", new_distance_sq.evalf())
    # print("old volume: ", old_distance_sq)
    # print('change in volume: ', new_distance_sq.evalf() - old_distance_sq)
    return new_distance_sq.evalf() - old_distance_sq

In [165]:
# bond_theta_soln = {0: 0, 1: sp.pi}
# bond_theta_soln = {0: sp.pi/4}
# bond_theta_soln = {0: 2*sp.pi/4}
# bond_theta_soln = {0: 3*sp.pi/4}
# bond_theta_soln = {0: 4*sp.pi/4}
# bond_theta_soln = {0: 5*sp.pi/4}
# bond_theta_soln = {0: 6*sp.pi/4}
# bond_theta_soln = {0: 7*sp.pi/4}

In [166]:
new_coords = compute_new_coords(bond_theta_soln)
volume_change = find_volume_change(coords_mol, new_coords)
print('change in volume: ', volume_change)

change in volume:  1333.14391192819


In [167]:
# brute force solution
start = time.time()
thetas = generate_thetas()
best_volume_change = 0
best_soln = {}
for i in range(8):
    for j in range(8):
        bond_theta_soln = {0: thetas[i], 1: thetas[j]}
        new_coords = compute_new_coords(bond_theta_soln)
        vol_difference = find_volume_change(coords_mol, new_coords)
        if vol_difference > best_volume_change:
            best_volume_change = vol_difference
            best_soln = bond_theta_soln
end = time.time()
print("The time of execution: ", (end-start), "s")    
print('best solution: ', best_soln)
print('best volume change: ', best_volume_change)




The time of execution:  18.05426335334778 s
best solution:  {0: 7*pi/4, 1: 3*pi/2}
best volume change:  1728.86274523471


In [94]:
from rdkit.Geometry import Point3D
conf = mol.GetConformer()
for i in range(len(coords_mol)):
    x, y, z = new_coords[i]
    conf.SetAtomPosition(i, Point3D(float(x), float(y), float(z)))

In [95]:
mblock = Chem.MolToMolBlock(mol)
view = py3Dmol.view(width=500, height=250)
view.addModel(mblock, 'mol')
view.setStyle({'stick':{}})
view.zoomTo()
view.show()

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

In [96]:
mol_file = './Molecules/25A12Rot_2.mol2'
mol = Chem.MolFromMol2File(mol_file)
mblock = Chem.MolToMolBlock(mol)
view = py3Dmol.view(width=500, height=250)
view.addModel(mblock, 'mol')
view.setStyle({'stick':{}})
view.zoomTo()
view.show()

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol